<a href="https://colab.research.google.com/github/kylehuynh205/kijiji-ML/blob/main/Big_Data_Management_and_Tools_Group_Project_Group_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install sparksql-magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=f0ce4d6759ae6c3693648d775604bf7dfe9504843a547e0436404df46192d59e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.2 MB/s eta 0:00:00


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()

# import modules from pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

In [3]:
#spark = SparkSession.builder.appName("Group Assignment - Group 6").config('spark.driver.memory', '4g').getOrCreate()
spark = SparkSession.builder \
.appName('Group Assignment - Group 6') \
.master('local[*]') \
.config('spark.sql.execution.arrow.pyspark.enabled', True) \
.config('spark.sql.session.timeZone', 'UTC') \
.config('spark.driver.memory','32G') \
.config('spark.ui.showConsoleProgress', True) \
.config('spark.sql.repl.eagerEval.enabled', True) \
.getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
sqlContext = SQLContext(spark)
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [4]:
# Download the Parquet file
!wget -q https://github.com/kylehuynh205/kijiji-ML/raw/main/data_parquet.parquet

# Read the Parquet file
kijiji = spark.read.parquet('data_parquet.parquet')

# Show the DataFrame
kijiji.show()

+---------+-------------+-----------+-----------+--------------------+-----------+---------+---------+----------+-----------------+--------------------+------------+-------------+---------------+-----------------+-------------+-----------+-----------------+-------------+---------+-----------+-----------+---------+---+-----+--------------------+---------------+---------------+-----------------+--------------+------------+
|  stock_s|vehicletype_s|bluetooth_b|forsaleby_s|      carprooflink_s|noofdoors_s|carmake_s|sunroof_b|carcolor_s|carmileageinkms_i|           cartrim_s|drivetrain_s|alloywheels_b|     carmodel_s|cartransmission_s|carbodytype_s|navsystem_b|pushbuttonstart_b|carfueltype_s|caryear_i|noofseats_s|date_posted|      amt|cur|    t|                 url|parkingassist_b|cruisecontrol_b|airconditioning_b|trailerhitch_b|cpoprogram_s|
+---------+-------------+-----------+-----------+--------------------+-----------+---------+---------+----------+-----------------+-------------------

In [ ]:
# Mount your Google Drive
# note that you will be asked to grant Google Colab access to your Google Drive
# in order to access the files
#from google.colab import drive
#drive.mount('/content/drive')

# load the json file to a Spark dataframe
# if you placed the data file in another location on your drive, you'll need to edit the path below
#file_type = "json"

# CSV options
#infer_schema = "false"
#first_row_is_header = "false"
#delimiter = ","

# Read The data
#kijiji = (spark.read
#  .option("header", "true")
#  .json("/content/drive/MyDrive/merged_data.json"))
#kijiji

In [5]:
#summary df before clean
kijiji.describe().toPandas()

,summary,stock_s,vehicletype_s,bluetooth_b,forsaleby_s,carprooflink_s,noofdoors_s,carmake_s,sunroof_b,carcolor_s,...,date_posted,amt,cur,t,url,parkingassist_b,cruisecontrol_b,airconditioning_b,trailerhitch_b,cpoprogram_s
0,count,207761,267573,183492,267573,130620,214216,267573,76517,242882,...,267563,256271,267600,267600,267615,84029,20227,21879,8599,10618
1,mean,2680439.9852428287,None,1.0,None,None,3.8542440857651696,None,1.0,None,...,None,3402758.1772771794,None,None,None,1.0,1.0,1.0,1.0,None
2,stddev,1.0303409924576253E7,None,0.0,None,None,0.5655565002508717,None,0.0,None,...,None,3171477.704785349,None,None,None,0.0,0.0,0.0,0.0,None
3,min,X60714,damaged,,delr,,,acura,,,...,2021-01-26,100.0,CAD,CONTACT,https://www.kijiji.ca/v-cars-trucks/abbotsford...,,,,,acura
4,max,x70233,used,true,ownr,https://vhr.carfax.ca/main?id=rJ0muyMMmzsVhU1/...,others,volvo,true,yellow,...,2024-07-21,5.99995E7,CAD,SWAP_TRADE,https://www.kijiji.ca/v-cars-trucks/woodstock-...,true,1,1,true,volkwagen


In [6]:
# check if any columns has NULL
from pyspark.sql.functions import col, when

for c in kijiji.columns:
    null_count = kijiji.where(col(c).isNull()).count()
    if null_count > 0:
        print(f"The column '{c}' has {null_count} NULL values.")

The column 'stock_s' has 59854 NULL values.
The column 'vehicletype_s' has 42 NULL values.
The column 'bluetooth_b' has 84123 NULL values.
The column 'forsaleby_s' has 42 NULL values.
The column 'carprooflink_s' has 136995 NULL values.
The column 'noofdoors_s' has 53399 NULL values.
The column 'carmake_s' has 42 NULL values.
The column 'sunroof_b' has 191098 NULL values.
The column 'carcolor_s' has 24733 NULL values.
The column 'carmileageinkms_i' has 42 NULL values.
The column 'cartrim_s' has 26621 NULL values.
The column 'drivetrain_s' has 25817 NULL values.
The column 'alloywheels_b' has 167333 NULL values.
The column 'carmodel_s' has 42 NULL values.
The column 'cartransmission_s' has 22519 NULL values.
The column 'carbodytype_s' has 42 NULL values.
The column 'navsystem_b' has 198978 NULL values.
The column 'pushbuttonstart_b' has 134213 NULL values.
The column 'carfueltype_s' has 24203 NULL values.
The column 'caryear_i' has 42 NULL values.
The column 'noofseats_s' has 71624 NULL 

In [7]:
# remove rows which all column are NULL, if there is
kijiji = kijiji.dropna(how='all')

# partition because the dataframe may be big
kijiji = kijiji.repartition(100)


In [8]:
kijiji.columns

['stock_s',
 'vehicletype_s',
 'bluetooth_b',
 'forsaleby_s',
 'carprooflink_s',
 'noofdoors_s',
 'carmake_s',
 'sunroof_b',
 'carcolor_s',
 'carmileageinkms_i',
 'cartrim_s',
 'drivetrain_s',
 'alloywheels_b',
 'carmodel_s',
 'cartransmission_s',
 'carbodytype_s',
 'navsystem_b',
 'pushbuttonstart_b',
 'carfueltype_s',
 'caryear_i',
 'noofseats_s',
 'date_posted',
 'amt',
 'cur',
 't',
 'url',
 'parkingassist_b',
 'cruisecontrol_b',
 'airconditioning_b',
 'trailerhitch_b',
 'cpoprogram_s']

* airconditioning_b: A boolean indicating if the car has air conditioning.
* alloywheels_b: A boolean indicating if the car has alloy wheels.
* amt: The amount or price of the car.
* bluetooth_b: A boolean indicating if the car has Bluetooth.
* carbodytype_s: The body type of the car (e.g., sedan, suvcrossover).
* carcolor_s: The color of the car.
* carfueltype_s: The fuel type of the car (e.g., gas).
* carmake_s: The make of the car (e.g., kia, honda).
* carmileageinkms_i: The mileage of the car in kilometers.
* carmodel_s: The model of the car (e.g., forte, hrv).
* carprooflink_s: The link to the CarProof report for the car.
* cartransmission_s: The transmission of the car (e.g., 2).
* cartrim_s: The trim of the car.
* caryear_i: The year of the car.
* cpoprogram_s: The certified pre-owned program of the car.
* cruisecontrol_b: A boolean indicating if the car has cruise control.
* cur: The currency of the price (e.g., CAD).
* date_posted: The date the car was posted for sale.
* drivetrain_s: The drivetrain of the car (e.g., fwd, awd).
* forsaleby_s: Who the car is for sale by (e.g., delr, ownr).
* navsystem_b: A boolean indicating if the car has a navigation system.
* noofdoors_s: The number of doors on the car.
* noofseats_s: The number of seats in the car.
* parkingassist_b: A boolean indicating if the car has parking assist.
* pushbuttonstart_b: A boolean indicating if the car has push button start.
* stock_s: The stock number of the car.
* sunroof_b: A boolean indicating if the car has a sunroof.
* t: The type of listing (e.g., FIXED, CONTACT).
* trailerhitch_b: A boolean indicating if the car has a trailer hitch.
* url: The URL of the listing.
* vehicletype_s: The type of vehicle (e.g., used, new).

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType
from pyspark.sql.functions import to_date

# clean up the data
kijiji = kijiji.withColumn('airconditioning_b', F.when(F.col('airconditioning_b').isNull(), False).otherwise(True))

kijiji = kijiji.withColumn('alloywheels_b', F.when(F.col('alloywheels_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('bluetooth_b', F.when(F.col('bluetooth_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('sunroof_b', F.when(F.col('sunroof_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('parkingassist_b', F.when(F.col('parkingassist_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('cruisecontrol_b', F.when(F.col('cruisecontrol_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('pushbuttonstart_b', F.when(F.col('pushbuttonstart_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('trailerhitch_b', F.when(F.col('trailerhitch_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('navsystem_b', F.when(F.col('navsystem_b').isNull(), False).otherwise(True))


kijiji = kijiji.withColumn('amt', F.when(F.col('amt').isNull(), 0.0).otherwise(F.col('amt')))
kijiji = kijiji.withColumn("amt", F.col("amt").cast(DecimalType(10, 2)))

kijiji = kijiji.withColumn('noofdoors_s', F.when(F.col('noofdoors_s').isNull(), 0).otherwise(F.col('noofdoors_s').cast("string")))
kijiji = kijiji.withColumn('noofseats_s', F.when(F.col('noofseats_s').isNull(), 0).otherwise(F.col('noofseats_s').cast("string")))
kijiji = kijiji.withColumn('carmileageinkms_i', F.when(F.col('carmileageinkms_i').isNull(), 0).otherwise(F.col('carmileageinkms_i').cast("int")))
kijiji = kijiji.withColumn('caryear_i', F.when(F.col('caryear_i').isNull(), -1).otherwise(F.col('caryear_i').cast("int")))



kijiji = kijiji.withColumn('carbodytype_s', F.when(F.col('carbodytype_s').isNull(), "Not Specified").otherwise(F.col('carbodytype_s')))
kijiji = kijiji.withColumn('carcolor_s', F.when(F.col('carcolor_s').isNull(), "Not Specified").otherwise(F.col('carbodytype_s')))
kijiji = kijiji.withColumn('carfueltype_s', F.when(F.col('carfueltype_s').isNull(), "Not Specified").otherwise(F.col('carfueltype_s')))
kijiji = kijiji.withColumn('carmake_s', F.when(F.col('carmake_s').isNull(), "Not Specified").otherwise(F.col('carmake_s')))
kijiji = kijiji.withColumn('carmodel_s', F.when(F.col('carmodel_s').isNull(), "Not Specified").otherwise(F.col('carmodel_s')))
kijiji = kijiji.withColumn('cartransmission_s', F.when(F.col('cartransmission_s').isNull(), "Not Specified").otherwise(F.col('cartransmission_s')))
kijiji = kijiji.withColumn('vehicletype_s', F.when(F.col('vehicletype_s').isNull(), "Not Specified").otherwise(F.col('vehicletype_s')))

kijiji = kijiji.withColumn('carprooflink_s', F.when(F.col('carprooflink_s').isNull(), "Not Available").otherwise(F.col('carprooflink_s')))
kijiji = kijiji.withColumn('cartrim_s', F.when(F.col('cartrim_s').isNull(), "Not Specified").otherwise(F.col('cartrim_s')))
kijiji = kijiji.withColumn('carprooflink_s', F.when(F.col('carprooflink_s').isNull(), "Not Available").otherwise(F.col('carprooflink_s')))
kijiji = kijiji.withColumn('cpoprogram_s', F.when(F.col('cpoprogram_s').isNull(), "Not Specified").otherwise(F.col('cpoprogram_s')))
kijiji = kijiji.withColumn('drivetrain_s', F.when(F.col('drivetrain_s').isNull(), "Not Specified").otherwise(F.col('drivetrain_s')))
kijiji = kijiji.withColumn('cur', F.when(F.col('cur').isNull(), "CAD").otherwise(F.col('cur')).cast("string"))


# to confirm:
kijiji = kijiji.withColumn('t', F.when(F.col('t').isNull(), "Not Specified").otherwise(F.col('t')))
kijiji = kijiji.withColumn('forsaleby_s', F.when(F.col('forsaleby_s').isNull(), "Not Specified").otherwise(F.col('forsaleby_s')))
kijiji = kijiji.withColumn('date_posted', F.when(F.col('date_posted').isNull(), "Not Specified").otherwise(F.col('date_posted')))
#kijiji = kijiji.withColumn("date_posted", to_date(kijiji.date_posted, 'MM/dd/yyyy'))
kijiji = kijiji.withColumn('stock_s', F.when(F.col('stock_s').isNull(), "Not Specified").otherwise(F.col('stock_s')))


In [10]:
# check if any columns has NULL
from pyspark.sql.functions import col, when

for c in kijiji.columns:
    null_count = kijiji.where(col(c).isNull()).count()
    if null_count > 0:
        print(f"The column '{c}' has {null_count} NULL values.")

In [11]:
#count after cleaning
kijiji.count()

267615

In [12]:
# convert to Pandas to read
df = kijiji.toPandas()
df

,stock_s,vehicletype_s,bluetooth_b,forsaleby_s,carprooflink_s,noofdoors_s,carmake_s,sunroof_b,carcolor_s,carmileageinkms_i,...,date_posted,amt,cur,t,url,parkingassist_b,cruisecontrol_b,airconditioning_b,trailerhitch_b,cpoprogram_s
0,Not Specified,used,False,ownr,Not Available,0,chevrolet,False,Not Specified,235000,...,2024-07-20,555500.00,CAD,FIXED,https://www.kijiji.ca/v-cars-trucks/saskatoon/...,False,False,False,False,Not Specified
1,24040910,used,True,delr,https://vhr.carfax.ca/?id=hCN2zpAyVBcSGr9JW7G4...,4,bmw,True,suvcrossover,61146,...,2024-04-13,6049900.00,CAD,FIXED,https://www.kijiji.ca/v-cars-trucks/calgary/20...,True,False,False,False,Not Specified
2,441355,used,True,delr,https://vhr.carfax.ca/?id=3iKFRKVPO2qydiaujMzL...,4,mazda,False,vanminicomma,80360,...,2024-07-17,1549500.00,CAD,FIXED,https://www.kijiji.ca/v-cars-trucks/hamilton/2...,False,False,False,False,Not Specified
3,441355,used,True,delr,https://vhr.carfax.ca/?id=3iKFRKVPO2qydiaujMzL...,4,mazda,False,vanminicomma,80360,...,2024-07-17,1549500.00,CAD,FIXED,https://www.kijiji.ca/v-cars-trucks/hamilton/2...,False,False,False,False,Not Specified
4,UT332687,used,False,delr,https://vhr.carfax.ca/?id=Wx5/Ee02RwM4HIBftPus...,2,gmc,False,pickuptruck,195081,...,2024-06-06,1719200.00,CAD,FIXED,https://www.kijiji.ca/v-cars-trucks/chilliwack...,False,False,False,False,Not Specified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267610,26535,used,True,delr,https://vhr.carfax.ca/?id=jnUHddqpiUy0dIpOzP8C...,4,ford,False,suvcrossover,30923,...,2023-11-05,3399700.00,CAD,FIXED,https://www.kijiji.ca/v-cars-trucks/winnipeg/2...,False,False,False,False,Not Specified
267611,Not Specified,used,False,delr,Not Available,0,othrmake,False,Not Specified,315,...,2024-07-20,6000000.00,CAD,FIXED,https://www.kijiji.ca/v-cars-trucks/ottawa/int...,False,False,False,False,Not Specified
267612,Not Specified,used,True,delr,Not Available,4,jaguar,True,sedan,135000,...,2024-02-21,2199500.00,CAD,FIXED,https://www.kijiji.ca/v-cars-trucks/regina/201...,True,False,False,False,Not Specified
267613,03102,used,True,delr,Not Available,4,jeep,False,suvcrossover,100609,...,2023-12-09,1799500.00,CAD,FIXED,https://www.kijiji.ca/v-cars-trucks/ottawa/201...,False,False,False,False,Not Specified


In [13]:
# get describe after clean
df_info = kijiji.describe().toPandas()
df_info

,summary,stock_s,vehicletype_s,forsaleby_s,carprooflink_s,noofdoors_s,carmake_s,carcolor_s,carmileageinkms_i,cartrim_s,...,carbodytype_s,carfueltype_s,caryear_i,noofseats_s,date_posted,amt,cur,t,url,cpoprogram_s
0,count,267615,267615,267615,267615,267615,267615,267615,267615,267615,...,267615,267615,267615,267615,267615,267615,267615,267615,267615,267615
1,mean,2680439.9852428283,None,None,None,3.084367004324211,None,None,96124.22959849038,27796.315282051328,...,None,None,2017.661431534107,3.725390603019788,None,3258517.799260,None,None,None,None
2,stddev,1.0303409924576256E7,None,None,None,1.621898910803407,None,None,84266.33424504065,77101.00868619164,...,None,None,25.986860054164854,2.391995792770907,None,3178351.498018389,None,None,None,None
3,min,X60714,Not Specified,Not Specified,,,Not Specified,Not Specified,0,,...,Not Specified,,-1,,2021-01-26,0.00,CAD,CONTACT,https://www.kijiji.ca/v-cars-trucks/abbotsford...,Not Specified
4,max,x70233,used,ownr,https://vhr.carfax.ca/main?id=rJ0muyMMmzsVhU1/...,others,volvo,wagon,2700000,Édition Wolfsburg,...,wagon,other,2025,other,Not Specified,59999500.00,CAD,SWAP_TRADE,https://www.kijiji.ca/v-cars-trucks/woodstock-...,volkwagen
